In [ ]:
import torch
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
import os
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryPrecisionRecallCurve

def clip_pred(imgs):
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    inputs = processor(
        text=["a synthetic image of a matrix", "a real image of a matrix"],
        images=imgs,
        return_tensors="pt",
        padding=True
    )

    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
    prob = logits_per_image.softmax(dim=1)  
    return prob

In [114]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [115]:
def metrics(ys, ts):
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    prc= BinaryPrecisionRecallCurve()
    acc.update(ys, ts)
    f1.update(ys, ts)
    cm.update(ys, ts)
    prc.update(ys, ts)
    return acc.compute(), f1.compute(), cm.compute(), prc.compute()




In [117]:
folder_path = r'C:\Users\vikto\Documents\GitHub\DTU_repo\deep_learning\4_Convolutional\images'
images = load_images_from_folder(folder_path)
probs = clip_pred(images)
labels = torch.tensor([1,1])
metrics(probs[:,1], labels)

(tensor(0.),
 tensor(0.),
 tensor([[0., 0.],
         [2., 0.]]),
 (tensor([1., 1., 1.]),
  tensor([1.0000, 0.5000, 0.0000]),
  tensor([0.1286, 0.3222])))